# Day 10

Imports.

In [1]:
from collections import namedtuple
from typing import Literal, Optional


Read input.

In [2]:
with open("10_input.txt", "r") as f:
    puzzle_data = f.read().splitlines()


Define data structures and utilities.

In [3]:
Pipe      = Literal["|", "-", "L", "J", "7", "F", ".", "S"]
Direction = Literal["up", "down", "right", "left"]
Position  = namedtuple("Position", "x y")

class DirectionError(Exception):
    def __init__(self, direction: Direction, next_pipe: Pipe) -> None:
        super().__init__(f"Going {direction} to {next_pipe} is not possible")

class PipeMaze:

    def __init__(self, maze: list[str]) -> None:
        self.maze: tuple[str] = maze
        self.n_rows: int = len(self.maze)
        self.n_cols: int = len(self.maze[0])
        self.curr_position: Position = self.get_starting_position()
        self.pipe_valid_directions = {
            "|": ("up", "down"),
            "-": ("right", "left"),
            "L": ("down", "left"),
            "J": ("down", "right"),
            "7": ("up", "right"),
            "F": ("up", "left"),
            "S": self._get_starting_position_valid_directions(),
        }

    def get_starting_position(self) -> Position:
        starting_1D_x = "".join(self.maze).find("S")
        starting_2D_x = starting_1D_x // self.n_cols
        starting_2D_y = starting_1D_x - self.n_cols * starting_2D_x
        return Position(x=starting_2D_x, y=starting_2D_y)

    def _get_starting_position_valid_directions(self) -> tuple[Direction]:
        all_directions  = ["up", "down", "right", "left"]
        all_valid_pipes = ["|7F", "|JL", "-7J", "-FL"]
        valid_directions = []

        for direction, valid_pipes in zip(all_directions, all_valid_pipes):
            next_position = self.get_next_position(direction=direction)
            next_pipe = self.get_pipe(position=next_position)
            if next_pipe in valid_pipes:
                valid_directions.append(direction)

        return tuple(valid_directions)

    def go_direction(self, direction: Direction) -> None:
        next_position = self.get_next_position(direction=direction)

        # Check if next_position is valid
        next_pipe = self.get_pipe(position=next_position)
        if direction not in self.pipe_valid_directions[next_pipe]:
            raise DirectionError(direction=direction, next_pipe=next_pipe)

        self.curr_position = next_position

    def get_next_position(self, direction: Direction) -> Position:
        next_position_x = self.curr_position.x
        next_position_y = self.curr_position.y
        if direction == "up":
            next_position_x -= 1
        elif direction == "down":
            next_position_x += 1
        elif direction == "right":
            next_position_y += 1
        elif direction == "left":
            next_position_y -= 1
        return Position(x=next_position_x, y=next_position_y)

    def get_pipe(self, position: Optional[Position] = None) -> Pipe:
        if position is None:
            position = self.curr_position
        return self.maze[position.x][position.y]

    def get_next_direction(self, prev_dir: Direction) -> Optional[Direction]:
        curr_pipe = self.get_pipe()
        match curr_pipe:
            case "|" | "-":
                next_dir = prev_dir
            case "L":
                next_dir = "right" if prev_dir == "down" else "up"
            case "J":
                next_dir = "up" if prev_dir == "right" else "left"
            case "7":
                next_dir = "down" if prev_dir == "right" else "left"
            case "F":
                next_dir = "down" if prev_dir == "left" else "right"
            case "S":
                return None
            case _:
                raise ValueError(f"Unknown pipe: {curr_pipe}")
        return next_dir

# Use the Shoelace formula to calculate the interior area
def calc_interior_area(positions: list[Position]) -> float:
    interior_area = 0
    for ix in range(len(positions) - 1):
        interior_area += positions[ix].x * positions[ix + 1].y
        interior_area -= positions[ix].y * positions[ix + 1].x

    interior_area += positions[-1].x * positions[0].y
    interior_area -= positions[-1].y * positions[0].x
    interior_area = abs(interior_area) / 2
    return interior_area


## Part 1

In [4]:
%%timeit

maze = PipeMaze(maze=puzzle_data)
starting_position = maze.curr_position

# First move from the starting point
initial_direction = maze.pipe_valid_directions["S"][0]
maze.go_direction(direction=initial_direction)
next_direction = maze.get_next_direction(prev_dir=initial_direction)

# Get loop length
loop_length = 1
while maze.curr_position != starting_position:
    loop_length += 1
    maze.go_direction(direction=next_direction)
    prev_direction = next_direction
    next_direction = maze.get_next_direction(prev_dir=prev_direction)

# Calculate how many steps its take to get from the loop starting position to
# the loop farthest point
n_steps_to_farthest = loop_length / 2
assert n_steps_to_farthest == 6_786


8.12 ms ± 35.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Part 2

In [5]:
%%timeit

maze = PipeMaze(maze=puzzle_data)
starting_position = maze.curr_position

# First move from the starting point
initial_direction = maze.pipe_valid_directions["S"][0]
maze.go_direction(direction=initial_direction)

# Get loop tiles positions
next_direction = initial_direction
positions = [starting_position]
while maze.curr_position != starting_position:
    positions.append(maze.curr_position)
    maze.go_direction(direction=next_direction)
    next_direction = maze.get_next_direction(prev_dir=next_direction)

# Calculate how many tiles are enclosed by the loop using the Shoelace formula
# and Pick's theorem
interior_area = calc_interior_area(positions=positions)
n_interior_tiles = int(abs(interior_area) - 0.5 * len(positions) + 1)
assert n_interior_tiles == 495


9.45 ms ± 55 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
